In [21]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig

In [22]:
raw = scipy.io.loadmat('./datasets/raw_training_data.mat')
data_glove_1 = raw['train_dg'][0][0]
data_glove_1 = np.delete(data_glove_1, 3, 1)
data_glove_2 = raw['train_dg'][1][0]
data_glove_2 = np.delete(data_glove_2, 3, 1)
data_glove_3 = raw['train_dg'][2][0]
data_glove_3 = np.delete(data_glove_3, 3, 1)

ecog_1 = raw['train_ecog'][0][0]
ecog_2 = raw['train_ecog'][1][0]
ecog_3 = raw['train_ecog'][2][0]

labels_1 = np.argmax(data_glove_1, axis=1)
labels_2 = np.argmax(data_glove_2, axis=1)
labels_3 = np.argmax(data_glove_3, axis=1)




In [23]:
data_glove_1[0]

array([-0.11974525, -0.36549759, -0.73717308, -0.71658897])

In [24]:
data_glove_1[120]

array([-0.11124992, -0.12539196, -0.66111088, -0.0558033 ])

In [25]:
data_glove_1[39] != data_glove_1[20]

array([False, False, False, False])

In [26]:
data_glove_1[20]

array([-0.11974525, -0.36549759, -0.73717308, -0.71658897])

In [27]:
for i in range(data_glove_1.shape[0] // 20):
    if (data_glove_1[i * 20] != data_glove_1[(i + 1) * 20 - 1]).sum() != 0:
        print(i * 20, data_glove_1[i * 20])

251000 [-0.70422745 -0.41832256 -0.25314713 -0.05946064]
251020 [-0.70582008 -0.41832256 -0.25314713 -0.05946064]
251040 [-0.70691967 -0.41834736 -0.25316048 -0.05946064]
251060 [-0.70729828 -0.41853237 -0.25326061 -0.05946064]
251080 [-0.70690155 -0.41893959 -0.25346661 -0.05949497]
251100 [-0.70566463 -0.41970539 -0.25378036 -0.05974483]
251120 [-0.70373344 -0.42082214 -0.25086975 -0.06031418]
251140 [-0.70103455 -0.42167664 -0.22680283 -0.06147766]
251160 [-0.69759178 -0.42181873 -0.16998196 -0.06352234]
251180 [-0.69319057 -0.42034721 -0.04553223 -0.06707954]
251200 [-0.68790054 -0.41644859  0.16960621 -0.07227898]
251220 [-0.68174076 -0.40971947  0.49785137 -0.07946301]
251240 [-0.67487812 -0.40061951  0.93284512 -0.08860493]
251260 [-0.66767788 -0.38940525  1.4420557  -0.09925365]
251280 [-0.66064167 -0.37688065  1.97132492 -0.11057949]
251300 [-0.65432835 -0.36292839  2.46135616 -0.12124157]
251320 [-0.64906025 -0.34725857  2.86704826 -0.12986088]
251340 [-0.6448822  -0.32957554

In [28]:
# labels[520:2020][labels[520:2020] != 1]
# np.where(labels[520:2020] != 1)

In [29]:
def filter_data(raw_eeg, fs=1000):
    """
    Write a filter function to clean underlying data.
    Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
    Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
    distorting the underlying data!

    Input: 
        raw_eeg (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
    Output: 
        clean_data (samples x channels): the filtered signal
    """
    dim = 100
    b = sig.firwin(numtaps=dim + 1, cutoff=[0.15, 200], pass_zero='bandpass', fs=fs)

    filtered_eeg = sig.filtfilt(b, 1, x=raw_eeg, axis=0)
    
    return filtered_eeg

In [30]:
train_test_ratio = 0.6

ecog_1_train = ecog_1[:int(train_test_ratio * ecog_1.shape[0])]
ecog_1_test = ecog_1[int(train_test_ratio * ecog_1.shape[0]):]
data_glove_1_train = data_glove_1[:int(train_test_ratio * data_glove_1.shape[0])]
data_glove_1_test = data_glove_1[int(train_test_ratio * data_glove_1.shape[0]):]

ecog_2_train = ecog_2[:int(train_test_ratio * ecog_2.shape[0])]
ecog_2_test = ecog_2[int(train_test_ratio * ecog_2.shape[0]):]
data_glove_2_train = data_glove_2[:int(train_test_ratio * data_glove_2.shape[0])]
data_glove_2_test = data_glove_2[int(train_test_ratio * data_glove_2.shape[0]):]

ecog_3_train = ecog_3[:int(train_test_ratio * ecog_3.shape[0])]
ecog_3_test = ecog_3[int(train_test_ratio * ecog_3.shape[0]):]
data_glove_3_train = data_glove_3[:int(train_test_ratio * data_glove_3.shape[0])]
data_glove_3_test = data_glove_3[int(train_test_ratio * data_glove_3.shape[0]):]


print(f"""
Before splitting: {ecog_1.shape[0]} samples
After splitting: {ecog_1_train.shape[0]} training samples and {ecog_1_test.shape[0]} testing samples
""")


Before splitting: 300000 samples
After splitting: 180000 training samples and 120000 testing samples



In [31]:
def NumWins(x, fs, winLen, winDisp):
    return int(1 + (x.shape[0] - winLen * fs) / (winDisp * fs))

winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap
NumWins(ecog_1, 1000, winLen, winDisp)

5999

In [32]:
def LineLength(x):
    return np.abs(np.diff(x, axis=0)).sum(axis=0)

def Area(x):
    return np.abs(x).sum(axis=0)

def Energy(x):
    return (x ** 2).sum(axis=0)

def ZeroCrossingMean(x):
    return ((x < x.mean(axis=0))[1:] & (x[:-1] > x.mean(axis=0)) | (x > x.mean(axis=0))[1:] & (x[:-1] < x.mean(axis=0))).sum(axis=0)

def numSpikes(x):
    #TODO: implement
    sig.find_peaks(x, height=0, distance=100)
    pass

def averageTimeDomain(x):
    #TODO: implement
    return np.mean(x, axis=0)

def averageFreqDomain(x):
    #TODO: implement
    fourier = np.fft.fft(x, axis=1)
    N = len(fourier)
    n = np.arange(N)
    T = N/1000
    freq = n/T 
    return [np.abs(fourier[(freq >= 8) & (freq <= 12)]).mean(axis=0), 
            np.abs(fourier[(freq >= 18) & (freq <= 24)]).mean(axis=0),
            np.abs(fourier[(freq >= 75) & (freq <= 115)]).mean(axis=0),
            np.abs(fourier[(freq >= 125) & (freq <= 159)]).mean(axis=0),
            np.abs(fourier[(freq >= 159) & (freq <= 175)]).mean(axis=0)]

def bandpower(x, fs, fmin, fmax):
    fs = 1000
    # win = 4 * sf
    freqs, psd = sig.welch(x, fs, axis=0, nperseg=x.shape[0])
    
    # Define delta lower and upper limits
    # fmin, fmax = 0.5, 4

    # Find intersecting values in frequency vector
    idx_delta = np.logical_and(freqs >= fmin, freqs <= fmax)
    
    from scipy.integrate import simps

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]  # = 1 / 4 = 0.25

    # Compute the absolute power by approximating the area under the curve
    delta_power = simps(psd[idx_delta], dx=freq_res, axis=0)
    
    return delta_power
    

def get_features(filtered_window, fs=1000):
    """
        Write a function that calculates features for a given filtered window. 
        Feel free to use features you have seen before in this class, features that
        have been used in the literature, or design your own!

        Input: 
        filtered_window (window_samples x channels): the window of the filtered ecog signal 
        fs: sampling rate
        Output:
        features (channels x num_features): the features calculated on each channel for the window
    """
    feat_LL = LineLength(filtered_window)
    feat_Area = Area(filtered_window)
    feat_Energy = Energy(filtered_window)
    feat_ZCM = ZeroCrossingMean(filtered_window)
    feat_TimeAvg = averageTimeDomain(filtered_window)
    feat_FreqAvg = averageFreqDomain(filtered_window)
    
    
    return np.hstack([#feat_LL, feat_Area, feat_Energy, feat_ZCM, 
                    #   feat_Energy,
                      feat_TimeAvg, 
                      bandpower(filtered_window, 1000, 8, 12),
                      bandpower(filtered_window, 1000, 18, 24),
                      bandpower(filtered_window, 1000, 75, 115),
                      bandpower(filtered_window, 1000, 125, 159),
                      bandpower(filtered_window, 1000, 159, 175)])

In [33]:
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
    """
        Write a function which processes data through the steps of filtering and
        feature calculation and returns features. Points will be awarded for completing
        each step appropriately (note that if one of the functions you call within this script
        returns a bad output, you won't be double penalized). Note that you will need
        to run the filter_data and get_features functions within this function. 

        Inputs:
        raw_eeg (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
        window_length: the window's length
        window_overlap: the window's overlap
        Output: 
        all_feats (num_windows x (channels x features)): the features for each channel for each time window
            note that this is a 2D array. 
    """
    raw_ecog = filter_data(raw_ecog, fs)
    
    window_disp = window_length - window_overlap
    
    all_feats = np.vstack([get_features(raw_ecog[int(i * window_disp * fs):int(i * window_disp * fs + window_length * fs), :], fs) for i in range(NumWins(raw_ecog, fs, window_length, window_overlap))])
    
    return all_feats

In [34]:
def create_R_matrix(features, N_wind):
    """ 
    Write a function to calculate the R matrix

    Input:
        features (samples (number of windows in the signal) x channels x features): 
        the features you calculated using get_windowed_feats
        N_wind: number of windows to use in the R matrix

    Output:
        R (samples x (N_wind*channels*features))
    """
    num_win = features.shape[0]
    num_channel_features = features.shape[1]
    
    # Append a copy of the first N-1 rows to the beginning of features
    features = np.vstack((features[:N_wind-1], features))
    
    R = np.zeros((num_win, N_wind * num_channel_features))
    
    for i in range(num_win):
        # Get the feature matrix for the current window
        # Resize the feature matrix and store in R
        R[i,:] = features[i:i+N_wind,:].reshape(-1)

    R = np.hstack((np.ones((R.shape[0], 1)), R))

    return R
    

In [35]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_1_train, 1000, winLen, winOverlap)
R_train = create_R_matrix(feature_train, 3)

feature_test = get_windowed_feats(ecog_1_test, 1000, winLen, winOverlap)
R_test = create_R_matrix(feature_test, 3)

# Downsample the glove data
Y_train = data_glove_1_train
Y_test = data_glove_1_test
Y_train = sig.resample(Y_train, R_train.shape[0], axis=0)
Y_test = sig.resample(Y_test, R_test.shape[0], axis=0)

In [36]:
# Compute the weights
f_train = np.linalg.pinv(R_train.T @ R_train) @ (R_train.T @ Y_train)

prediction_LR = (R_test @ f_train)

In [37]:
# # import and Do PCA
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled_R_train = scaler.fit_transform(R_train)
# scaled_R_test = scaler.transform(R_test)


# from sklearn.decomposition import PCA
# pca = PCA(n_components=500)
# R_train_pca = pca.fit_transform(scaled_R_train)
# R_test_pca = pca.transform(scaled_R_test)
# print(pca.explained_variance_ratio_.sum())

from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=100, max_depth=5, max_leaf_nodes=200, random_state=0)

rf_reg.fit(R_train, Y_train)

prediction_RF = rf_reg.predict(R_test)

rf_reg.score(R_test, Y_test)

0.07579937935040704

In [38]:
print('For Subject 1')
print(f'For linear regression: {[pearsonr(prediction_LR[:,i], Y_test[:,i]) for i in range(4)]}')
print(f'For random forest: {[pearsonr(prediction_RF[:,i], Y_test[:,i]) for i in range(4)]}')

For Subject 1
For linear regression: [PearsonRResult(statistic=0.4054090765756593, pvalue=1.3978331306633656e-95), PearsonRResult(statistic=0.5045790666973186, pvalue=3.691994182398747e-155), PearsonRResult(statistic=0.10661036535565127, pvalue=1.6594122665966994e-07), PearsonRResult(statistic=0.13956713354405995, pvalue=6.602394426642155e-12)]
For random forest: [PearsonRResult(statistic=0.35504247410779016, pvalue=3.3817087722696887e-72), PearsonRResult(statistic=0.5030072789456146, pvalue=4.713259277890041e-154), PearsonRResult(statistic=0.13972082082395523, pvalue=6.2582978237528185e-12), PearsonRResult(statistic=0.11589179787218815, pvalue=1.2514547149793966e-08)]


In [39]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_2_train, 1000, winLen, winOverlap)
R_train = create_R_matrix(feature_train, 3)

feature_test = get_windowed_feats(ecog_2_test, 1000, winLen, winOverlap)
R_test = create_R_matrix(feature_test, 3)

# Downsample the glove data
Y_train = data_glove_2_train
Y_test = data_glove_2_test
Y_train = sig.resample(Y_train, R_train.shape[0], axis=0)
Y_test = sig.resample(Y_test, R_test.shape[0], axis=0)

# Compute the weights
f_train = np.linalg.pinv(R_train.T @ R_train) @ (R_train.T @ Y_train)
prediction_LR = (R_test @ f_train)

from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=100, max_depth=5, max_leaf_nodes=200)
rf_reg.fit(R_train, Y_train)
prediction_RF = rf_reg.predict(R_test)


print('For Subject 2')
print(f'For linear regression: {[pearsonr(prediction_LR[:,i], Y_test[:,i]) for i in range(4)]}')
print(f'For random forest: {[pearsonr(prediction_RF[:,i], Y_test[:,i]) for i in range(4)]}')

For Subject 2
For linear regression: [PearsonRResult(statistic=0.3554504418924376, pvalue=2.2697551930662644e-72), PearsonRResult(statistic=0.17296469290372415, pvalue=1.442095527153836e-17), PearsonRResult(statistic=0.21153573413317261, pvalue=1.13399096689403e-25), PearsonRResult(statistic=0.15650212493551616, pvalue=1.2684556052592902e-14)]
For random forest: [PearsonRResult(statistic=0.37024683927915203, pvalue=7.970421299285082e-79), PearsonRResult(statistic=0.18646224505938816, pvalue=3.2930386465967293e-20), PearsonRResult(statistic=0.25629849443978914, pvalue=2.7216961090821537e-37), PearsonRResult(statistic=0.14264413140658905, pvalue=2.235752124616957e-12)]


In [40]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_3_train, 1000, winLen, winOverlap)
R_train = create_R_matrix(feature_train, 5)

feature_test = get_windowed_feats(ecog_3_test, 1000, winLen, winOverlap)
R_test = create_R_matrix(feature_test, 5)

# Downsample the glove data
Y_train = data_glove_3_train
Y_test = data_glove_3_test
Y_train = sig.resample(Y_train, R_train.shape[0], axis=0)
Y_test = sig.resample(Y_test, R_test.shape[0], axis=0)

# Compute the weights
f_train = np.linalg.pinv(R_train.T @ R_train) @ (R_train.T @ Y_train)
prediction_LR = (R_test @ f_train)

from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=100, max_depth=5, max_leaf_nodes=200)
rf_reg.fit(R_train, Y_train)
prediction_RF = rf_reg.predict(R_test)

print('For Subject 3')
print(f'For linear regression: {[pearsonr(prediction_LR[:,i], Y_test[:,i]) for i in range(4)]}')
print(f'For random forest: {[pearsonr(prediction_RF[:,i], Y_test[:,i]) for i in range(4)]}')

For Subject 3
For linear regression: [PearsonRResult(statistic=0.5356172949140474, pvalue=3.117861869322194e-178), PearsonRResult(statistic=0.32739792303867016, pvalue=4.7924348281116704e-61), PearsonRResult(statistic=0.333654309674209, pvalue=1.8015881771687087e-63), PearsonRResult(statistic=0.4005173912241265, pvalue=3.985776901194511e-93)]
For random forest: [PearsonRResult(statistic=0.6636942774044342, pvalue=1.3724215189454735e-304), PearsonRResult(statistic=0.4334640821086646, pvalue=1.7661376309971077e-110), PearsonRResult(statistic=0.5512218112162279, pvalue=8.975010639028507e-191), PearsonRResult(statistic=0.26902164285432106, pvalue=4.75532281052035e-41)]


In [41]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_1, 1000, winLen, winOverlap)
R_train = create_R_matrix(feature_train, 5)

# Downsample the glove data
Y_train = data_glove_1
Y_train = sig.resample(Y_train, R_train.shape[0], axis=0)

# Compute the weights
f_train = np.linalg.pinv(R_train.T @ R_train) @ (R_train.T @ Y_train)

from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=100, max_depth=5, max_leaf_nodes=200)
rf_reg.fit(R_train, Y_train)


np.save('./models/LR_Matrix_S1', f_train)
pickle.dump(rf_reg, open('./models/RF_Matrix_S1.pth', 'wb'))

In [42]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_2, 1000, winLen, winOverlap)
R_train = create_R_matrix(feature_train, 5)

# Downsample the glove data
Y_train = data_glove_2
Y_train = sig.resample(Y_train, R_train.shape[0], axis=0)

# Compute the weights
f_train = np.linalg.pinv(R_train.T @ R_train) @ (R_train.T @ Y_train)

from sklearn. ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=100, max_depth=5, max_leaf_nodes=200)
rf_reg.fit(R_train, Y_train)


np.save('./models/LR_Matrix_S2', f_train)
pickle.dump(rf_reg, open('./models/RF_Matrix_S2.pth', 'wb'))

In [43]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_3, 1000, winLen, winOverlap)
R_train = create_R_matrix(feature_train, 5)

# Downsample the glove data
Y_train = data_glove_3
Y_train = sig.resample(Y_train, R_train.shape[0], axis=0)

# Compute the weights
f_train = np.linalg.pinv(R_train.T @ R_train) @ (R_train.T @ Y_train)

from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=100, max_depth=5, max_leaf_nodes=200)
rf_reg.fit(R_train, Y_train)


np.save('./models/LR_Matrix_S3', f_train)
pickle.dump(rf_reg, open('./models/RF_Matrix_S3.pth', 'wb'))